# **Out-of-Core Single-Cell Analysis with RAPIDS-SingleCell & Dask**  
**Author:** [Severin Dicks](https://github.com/Intron7), [T.J. Chen](https://github.com/tjchennv)

**Copyright** [scverse](https://scverse.org)

In this notebook, we demonstrate the **out-of-core computation** capabilities of **rapids-singlecell** using **Dask**.  
This approach allows us to analyze larger scale datasets, such as 1.3 million to **11 million cells** efficiently, even on relatively small hardware.  
The user also has the option to switch to 1.3 million cells (more suitable for a smaller instance).

By leveraging **Dask**, we can:  
- **Process large-scale single-cell datasets** without exceeding memory limits.  
- **Enable chunk-based execution**, loading only the necessary data into memory at any time.  

The notebook covers the full analysis pipeline:
1. **Preprocessing** — QC, filtering, normalization, HVG selection, scaling, and PCA
2. **Neighborhood Graph** — Using the `all_neighbors` algorithm for efficient approximate nearest neighbor search at scale
3. **Clustering & Visualization** — Leiden clustering and UMAP embedding
4. **Differential Expression** — Identifying marker genes per cluster with the GPU-accelerated `wilcoxon_binned` method

This method makes **large-scale single-cell analysis feasible** on standard hardware setups,  
removing barriers to working with massive datasets.


In [1]:
import dask
import time

from dask_cuda import LocalCUDACluster
from dask.distributed import Client

**RMM (RAPIDS Memory Manager)** controls how GPU memory is allocated during the analysis.

By default, memory requests go to the GPU driver, which can become a bottleneck when processing millions of cells. RMM **pre-allocates a large memory pool upfront** and serves requests from it internally, keeping allocations fast throughout the pipeline.

The `rmm_cupy_allocator` ensures that both RAPIDS and CuPy draw from the **same pool**, rather than maintaining separate ones that can fragment and cause out-of-memory errors mid-analysis.

> **In practice:** The pool sizes you set in the Dask cluster configuration above are what back this allocator, so these two setup steps work together.

In [ ]:
import rmm #RAPIDS Memory Manager (controls how GPU memory is allocated and managed)
import cupy as cp
import rapids_singlecell as rsc
from rmm.allocators.cupy import rmm_cupy_allocator #ensures that both RAPIDS and CuPy draw from the same memory pool

## **Auto-Selecting Dataset Based on Available GPU Memory**

This cell detects your GPU configuration and automatically selects the appropriate dataset:
- **< 96 GB** total GPU memory → 1.3M cell dataset (single GPU setups)
- **≥ 96 GB** total GPU memory → 11M cell dataset (multi-GPU setups)

It also sets `preprocessing_gpus` to use all available GPUs automatically.

In [ ]:
import pynvml

# Initialize NVML to query GPU hardware information
pynvml.nvmlInit()
n_gpus = pynvml.nvmlDeviceGetCount()

# Collect name and memory (in GB) for each available GPU
gpu_info = [
    (pynvml.nvmlDeviceGetName(pynvml.nvmlDeviceGetHandleByIndex(i)),
     pynvml.nvmlDeviceGetMemoryInfo(pynvml.nvmlDeviceGetHandleByIndex(i)).total / 1024**3)
    for i in range(n_gpus)
]

# Use GPU 0 memory as the reference — all GPUs on a given instance are typically identical,
# and this ensures dataset selection reflects true per-GPU capacity rather than aggregate totals
gpu0_memory_gb = gpu_info[0][1]

preprocessing_gpus = "all"  # Uses all available GPUs. To target specific devices, replace with a comma-separated index string (e.g. "0,1") — indices match the GPU numbers printed below

# Select dataset based on per-GPU memory
if gpu0_memory_gb < 24:
    url, output, final, dataset = None, None, None, None
    print(f"WARNING: GPU 0 has only {gpu0_memory_gb:.1f} GB of memory.")
    print("This notebook requires at least 24 GB of GPU memory to run.")
    print("Please use a larger instance (e.g. L40S 48 GB or higher) for best performance.")
elif gpu0_memory_gb >= 96:
    url    = "https://datasets.cellxgene.cziscience.com/3817734b-0f82-433b-8c38-55b214200fff.h5ad"
    output = "./h5/cell_atlas.h5ad"
    final  = "./zarr/cell_atlas.zarr"
    dataset = "11M Cell Atlas"
else:
    url    = "https://exampledata.scverse.org/rapids-singlecell/1M_brain_cells_10X.sparse.h5ad"
    output = "./h5/nvidia_1.3M.h5ad"
    final  = "./zarr/nvidia_1.3M.zarr"
    dataset = "1.3M Brain Cells"

# Print a summary of the detected hardware and selected configuration
gpu_rows = "\n".join(f"  GPU {i}: {name} ({mem:.1f} GB)" for i, (name, mem) in enumerate(gpu_info))
print(f"GPU Configuration\n{'-'*40}\n{gpu_rows}\n{'-'*40}\nDataset selected: {dataset}\nGPUs in use:      {preprocessing_gpus}")

## **Initializing a Dask Cluster for Out-of-Core Computation**  

To enable **out-of-core computation** and parallel processing,  
we initialize a **Dask CUDA cluster**, which distributes computations across available GPU resources.  

We create a **local GPU cluster** with the following configuration:  


In [ ]:
%%time
cluster = LocalCUDACluster(CUDA_VISIBLE_DEVICES=preprocessing_gpus,
                           threads_per_worker=10,
                           protocol="ucx",
                           rmm_pool_size= "10GB",                        # Pre-allocate GPU memory upfront to avoid repeated OS requests
                           rmm_maximum_pool_size = "40GB",                # Hard cap on pool growth — keep this <= your GPU's VRAM
                           rmm_allocator_external_lib_list= "cupy",       # Route CuPy allocations through RMM for unified memory management
                          )

client = Client(cluster)

client

In [4]:
import rapids_singlecell as rsc
import scanpy as sc
import anndata as ad

import sys
sys.path.insert(0, "./scripts")
from gpu_plotting import plot_umap

## **Loading Large Datasets into AnnData with Dask**  

To efficiently handle large-scale single-cell datasets, we load data directly from an **HDF5 (`h5`) or Zarr file**  
into an **AnnData object** using **Dask arrays**. This enables **lazy loading**, allowing data to be processed in chunks  
without exceeding memory limits.  

We achieve this using **`read_elem_lazy`**, which loads the expression matrix (`X`) as a **Dask array**

Let's first download the data. The dataset has been automatically selected based on your GPU memory in the configuration cell above.
- For the 1.3M cells dataset, `nvidia_1.3M`, this may take about 5 minutes, as it is 5.3 gigabytes to download and convert.
- For the 11M cells dataset, `cell_atlas`, downloading can get considerable time, as it is over 43GB of data to download and convert.

If you already have the data ready, it will skip the download

In [ ]:
import wget
import os
from anndata.experimental import read_elem_lazy
import h5py

data_dir = "./h5"

if not os.path.exists(final):
    if not os.path.exists(data_dir):
        print('creating data directory')
        os.system('mkdir ./h5')
    else:
        print(f'{data_dir} directory found')

    if not os.path.isfile(output):
        print(f'Downloading cell data into {output}...')
        wget.download(url, output)
    else:
        print(f'{output} dataset found')

    print(f'Converting {output} into {final}...')
    SPARSE_CHUNK_SIZE = 20_000

    f = h5py.File(output)
    X = f["X"]
    shape = X.attrs["shape"]
    adata = ad.AnnData(
        X = read_elem_lazy(X, (SPARSE_CHUNK_SIZE, shape[1])),
        obs = ad.io.read_elem(f["obs"]),
        var = ad.io.read_elem(f["var"]))
    f.close()

    adata.write_zarr(final)
    print(f'{final} is ready for use!')
else:
    print(f'{final} zarr dataset directory found')

Now, let's read in the zarr data.

In [6]:
import zarr

SPARSE_CHUNK_SIZE = 20_000

f = zarr.open(final)
X = f["X"]
shape = X.attrs["shape"]
adata = ad.AnnData(
    X = read_elem_lazy(X, (SPARSE_CHUNK_SIZE, shape[1])),
    obs = ad.io.read_elem(f["obs"]),
    var = ad.io.read_elem(f["var"]))



In [7]:
adata.var_names = adata.var.feature_name

## **Transferring AnnData to GPU for Accelerated Computation**  

Once the dataset is loaded as a **Dask-backed AnnData object**,  
we transfer it to the **GPU** to leverage **RAPIDS-SingleCell’s** accelerated processing.  

We use **`rsc.get.anndata_to_GPU()`**, which efficiently moves the AnnData object to GPU memory:  


In [8]:
rsc.get.anndata_to_GPU(adata)

In [9]:
adata.X

dask.array<X_to_GPU, shape=(11441407, 45854), dtype=float32, chunksize=(20000, 45854), chunktype=cupyx.csr_matrix>

## **Quality Control (QC) Metrics Calculation (Requires Synchronization)**  

Before proceeding with further analysis, we compute **quality control (QC) metrics**  
to assess dataset quality and filter out low-quality cells or genes.  

We use **`rsc.pp.calculate_qc_metrics()`** to calculate key QC metrics

Although we are working with Dask-backed AnnData, this operation requires a synchronization step.
This means that Dask computations must be evaluated immediately,
so the process is not completely lazy like other out-of-core operations.

In [10]:
%%time
rsc.pp.calculate_qc_metrics(adata)

CPU times: user 42.7 s, sys: 1.45 s, total: 44.1 s
Wall time: 44 s


## **Filtering Cells and Genes Without Additional Computation**  

Instead of using **`sc.pp.filter_cells`** and **`sc.pp.filter_genes`**,  
we apply filtering directly using boolean indexing to **avoid extra computation**.

**Why Use Direct Indexing Instead of Built-in Functions?**
* More Efficient with Dask → Avoids triggering additional computations.
* Preserves Lazy Execution → Filtering is applied without forcing full dataset evaluation.
* Copy is Essential → Using `.copy()` prevents views, which may not work reliably with Dask-backed AnnData.

In [11]:
adata = adata[(adata.obs["n_genes_by_counts"]<=10000) 
            & (adata.obs["n_genes_by_counts"]>=200)].copy()

In [12]:
adata.X

dask.array<getitem, shape=(11441244, 45854), dtype=float32, chunksize=(20120, 45854), chunktype=cupyx.csr_matrix>

## **Log Normalization (Fully Lazy Execution)**  

Next, we apply **log normalization** to scale gene expression values.  
This step ensures that differences in sequencing depth across cells do not dominate downstream analysis.  

In [13]:
%%time
rsc.pp.normalize_total(adata,target_sum = 10000)
rsc.pp.log1p(adata)

CPU times: user 5.56 ms, sys: 974 μs, total: 6.53 ms
Wall time: 5.5 ms


## **Selecting Highly Variable Genes (Requires Synchronization)**  

To focus on the most informative features, we identify **highly variable genes (HVGs)**  
using the **Cell Ranger** method and subset the dataset accordingly.  

**Important Considerations:**
* Requires Synchronization → Computing highly variable genes triggers evaluation,
meaning this step is not fully lazy when using Dask.
* Copy is Essential → Using `.copy()` prevents views, ensuring the operation works properly with Dask-backed AnnData.


In [14]:
%%time
rsc.pp.highly_variable_genes(adata,n_top_genes=5000, flavor="cell_ranger")
adata = adata[:,adata.var.highly_variable].copy()

/home/sdicks/micromamba/envs/rapids-26.02/lib/python3.13/site-packages/distributed/client.py:3375: UserWarning: Sending large graph of size 22.25 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


CPU times: user 1min, sys: 1.83 s, total: 1min 2s
Wall time: 55.9 s


## **Principal Component Analysis (PCA) on GPU (Two-Step Synchronization Process)**  

To reduce dimensionality while preserving meaningful variation,  
we perform **Principal Component Analysis (PCA)** using **GPU acceleration**.

Understanding the Two-Step Synchronization in PCA:
1. Mandatory Synchronization for Covariance & Mean Calculation
    * PCA requires computing the covariance matrix and mean vector,
    which must be explicitly synchronized before proceeding.
    * This step is handled automatically within `rsc.pp.pca()`.

2. Finalizing the Transformation with `.compute()`
    * After computing the principal components, the data remains lazy (Dask CuPy array).
    * Calling `.compute()` on `adata.obsm["X_pca"]` performs the final transformation,
      projecting the data onto the computed PCs and materializing the result as a fully computed CuPy array.

**Why This Matters?**
* The first synchronization (**covariance & mean**) ensures the PCA model is ready.
* The second synchronization (`compute()`) ensures that the transformed data is fully realized
for downstream analyses like clustering and UMAP.

In [16]:
%%time
rsc.pp.pca(adata, n_comps = 100,mask_var=None)
adata.obsm["X_pca"]=adata.obsm["X_pca"].compute()

CPU times: user 55.7 s, sys: 2.23 s, total: 57.9 s
Wall time: 57.8 s


## **Computing the Neighborhood Graph with `all_neighbors`**

Next, we compute the neighborhood graph using the **`all_neighbors`** algorithm.
Unlike brute-force search, which computes distances to every data point,
`all_neighbors` uses an approximate nearest neighbor approach optimized for large-scale datasets.

This algorithm is well-suited for out-of-core workflows where datasets can contain millions of cells,
as it provides a good balance between accuracy and computational efficiency on GPUs.

In [17]:
%%time
rsc.pp.neighbors(adata, n_neighbors=15, n_pcs=50, algorithm="all_neighbors")

CPU times: user 4min 7s, sys: 13 s, total: 4min 20s
Wall time: 29 s


## **UMAP Embedding**

We compute a **UMAP** (Uniform Manifold Approximation and Projection) embedding
to visualize the high-dimensional data in two dimensions.
The RAPIDS GPU implementation of UMAP is significantly faster than CPU-based methods,
making it practical even for datasets with millions of cells.

In [18]:
%%time
rsc.tl.umap(adata)

CPU times: user 20.2 s, sys: 4.33 s, total: 24.6 s
Wall time: 23.1 s


## **Leiden Clustering**

We use the **Leiden** algorithm for graph-based clustering.
Leiden is an improved version of the Louvain algorithm that guarantees well-connected, more stable clusters.
The RAPIDS implementation accelerates the clustering on GPUs.

In [19]:
%%time
rsc.tl.leiden(adata, resolution=0.6)

CPU times: user 15.5 s, sys: 5.06 s, total: 20.5 s
Wall time: 18.5 s


In [23]:
plot_umap(adata, color_key="leiden", background=None)

SyntaxError: unmatched ')' (2098790885.py, line 1)

**Freeing GPU Memory Before Continuing**

Unlike CPU RAM, **GPU memory is not automatically released** when analysis steps complete —
data from every preceding step (the full expression matrix, PCA, neighbor graph, UMAP embeddings)
remains loaded on the GPU until the kernel is explicitly shut down.

For an 11M cell dataset, this can consume most or all of available VRAM, leaving no room for
downstream steps in subsequent notebooks.

Restarting the kernel here is the cleanest way to guarantee a full reset of GPU memory.

> **Expected behavior:** You will see a "kernel died" or disconnection message — this is intentional.
> If you encounter a CUDA or Out-of-Memory (OOM) error at any earlier point in the notebook,
> restarting all kernels and starting fresh is the recommended first step.

In [ ]:
# These notebooks are very GPU memory intensive!
# You can comment this out if you want to continue exploring the notebook.
# Please consult the README for more tips and tricks.

import IPython

IPython.Application.instance().kernel.do_shutdown(True)